# Entendimento do Negócio

Este notebook tem como objetivo apresentar o contexto de negócio e a motivação para a análise e predição dos casos de fraude de cartões de crédito.

A facilidade de uso do cartão de crédito é visível e de senso comum para a maioria dos brasileiros. Com a crescente oferta de lojas onlines, programas de assinatura e benefícios atrelados ao uso do cartão, associar o uso do cartão de crédito ao dia a dia se torna quase institivo para segmentos da população. Um levantamento realizado em 2024 estimou-se que havia 212 milhões de cartões de crédito no brasil, sendo este número o dobro da população ocupada [^1]. Estimou-se também que cada brasileiro possui mais de um único cartão[^2], tais números somados a más práticas de segurança da informação realizadas, como vazamento de dados, compartilhamento e utilização de cartões em sites suspeitos alavanca o número de possibilidade de fraudes e golpes financeiros a serem realizados.

<center>
    <img src="../assets/man_with_card.jpg" width="300"/>
</center>

Em 2023 ocorreram 3,7 milhões de tentativas de fraude, totalizando em uma quantia de R$ 3,5 bilhões[^3]. Tais números acendem o alerta de como as empresas de cartão devem melhorar cada vez mais seus sistemas de detecção de fraudes. Os avanços da Inteligência Artificial e práticas de Machine Learning são de extrema valia para a redução desses números, garantindo segurança nas transações e evitando que os clientes sofram com tais tentativas caso estas sejam bem sucedidas.

[^1]: [País já tem mais que dois cartões de crédito para cada brasileiro que trabalha, O Globo, Acessado em 16/01/2025 às 21:09](https://example.com)\
[^2]: [29\% dos brasileiros têm cinco ou mais cartões, Exame, Acessado em 16/01/2025 às 21:09](https://exame.com/invest/minhas-financas/brasileiros-cinco-cartoes-credito/).\
[^3]: [Brasil registra R$ 3,5 bi em tentativas de fraude, Valor Investe, Acessado em 16/01/2025 às 21:09](https://valorinveste.globo.com/produtos/servicos-financeiros/noticia/2024/04/18/brasil-registra-r-35-bi-em-tentativas-de-fraude-com-cartao-de-credito-aponta-mapa-da-fraude.ghtml)

Assumindo que somos responsáveis pelo processo de prevenção a fraude de uma instituição financeira arbitrária, direcionamos nossa atenção para os dados que iremos utilizar neste projeto. Iremos realizar uma **análise estatística descritiva** dos dados e aplicá-los a modelos de **Machine Learning** para otimizar a identificação das tentativas de fraude.

## Premissas e descrição dos dados

É de grande importância listarmos as informações prévias que possuímos, tanto da descrição dos dados, quanto das premissas que iremos assumir. Tais informações são cruciais para nosso objetivo e consequentemente serão carregadas ao longo dos Notebooks Jupyter que compõem este projeto.

- score_1 a score_10: Representam notas de bureau de crédito (score de crédito) fornecidos pela empresa, usamos para identificar se o comprador é confiável, também podem haver outros dados anonimizados que não temos informação sobre
- pais: Em qual pais a compra foi realizada
- produto: Qual tipo de produto que está sendo comprado no e-commerce
- categoria_produto: categoria a qual esse produto se encaixa
- entrega_doc_1 a entrega_doc_3: Documentos que são requisitados no momento de abertura da conta.
- score_fraude_modelo: Score do modelo de predição atual e já implementado pela empresa. É a probabilidade da compra realizada ser uma fraude ou não. Varia de 0 a 100, quanto mais próximo de 100, maior a certeza que a compra é fraude
- fraude: Informação se aquela compra foi fraudulenta ou não. Foi rotulada após alguns dias para conferência do modelo atual.

**Importante**: Sabemos que a empresa ganha 10% do valor quando um pagamento é aprovado corretamente, porém a cada fraude aprovada perdemos todo (100%) do valor do pagamento. Iremos utilizar essas porcentagens para verificarmos como otimizar nossos ganhos e os KPIs interessantes para a análise.

## Avaliação das métricas atuais

Iremos utilizar nossa amostra atual para calcularmos os seguintes KPIs:
1. *Incoming Pressure*: Taxa de transações fraudulentas recebidas pela quantia total de transações 
2. Taxa de Aprovação: Proporção de transações aprovadas após análise 
3. Taxa de declínio: Proporção de transações rejeitadas após análise
4. Precisão: Proporção de compras ditas como fraudes que realmente são
5. Taxa de Detecção (Recall): Quantas fraudes reais o modelo identificou
6. Taxa de falsos positivos: Proporção de transações legítimas classificadas como fraudulentas

Tais taxas são relativas a esta amostra analisada e não necessariamente representam os valores da população de transações. 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../artifacts/data_ingestion/dados.csv")

In [4]:
df.columns

Index(['score_1', 'score_2', 'score_3', 'score_4', 'score_5', 'score_6',
       'pais', 'score_7', 'produto', 'categoria_produto', 'score_8', 'score_9',
       'score_10', 'entrega_doc_1', 'entrega_doc_2', 'entrega_doc_3',
       'data_compra', 'valor_compra', 'score_fraude_modelo', 'fraude'],
      dtype='object')

### Maximizar receita obtida e Limiar ótimo

Para verificar a situação atual do negócio vamos utilizar neste momento apenas as colunas `valor_compra`, `fraude` e `score_fraude_modelo`, assumindo a premissa de ganhos, vamos encontrar em primeiro momento o limiar do valor de `score_fraude_modelo` que traga o maior valor monetário no **modelo de predição atual**.

Iremos fazer apenas verificações simples nas colunas, para garantir que nenhum valor nulo interfira nos cálculos dos limiares.

In [5]:
# Verificando os valores que fraude assume na amostra
df["fraude"].value_counts()

fraude
0    142500
1      7500
Name: count, dtype: int64

In [6]:
# Verificando se há algum valor nulo na coluna de predição do modelo.
df["score_fraude_modelo"].isnull().sum()

0

In [7]:
def calculate_revenue(df, score_column, fraud_column, value_column, threshold):
    """
    Função que calcula faturamento da amostra a partir de um limiar de predição.
    O cálculo de faturamento se baseia nas premissas:
        10% do valor de compras aceitas é recebido.
        100% do valor de compras fraudulentar aceitas é perdido.

    Args:
    - df (pd.DataFrame): DataFrame com os dados de compras.
    - score_column (str): Nome da coluna do score de predição.
    - fraud_column (str): Nome da coluna classificação real da fraude.
    - value_column (str): Nome da coluna com o valor da compra.
    - threshold (int): Limiar de score para aceitação.

    Returns:
    - total_revenue (float): Faturamento total para o limiar fornecido.
    """

    accepted_sales = df.loc[df[score_column] < threshold]

    total_normal_accepted = accepted_sales.loc[
        accepted_sales[fraud_column] == 0, value_column
    ].sum()

    total_fraud_accepted = accepted_sales.loc[
        accepted_sales[fraud_column] == 1, value_column
    ].sum()

    # Multiplica-se por 0.1 pois apenas 10% do valor da compra é recebido.
    total_income = 0.1 * total_normal_accepted
    total_revenue = total_income - total_fraud_accepted

    return total_income, total_fraud_accepted, total_revenue


revenue_list = []
for threshold in range(1, 100):
    revenue_list.append(
        calculate_revenue(
            df, "score_fraude_modelo", "fraude", "valor_compra", threshold
        ),
    )

revenue_df = pd.DataFrame(revenue_list, columns=["income", "loss", "revenue"])
revenue_df
best_threshold = revenue_df["revenue"].idxmax()

best_record = revenue_df.iloc[best_threshold]

best_income = best_record["income"]
minimum_loss = best_record["loss"]
best_revenue = best_record["revenue"]

print(f"O limiar ótimo encontrado para a amostra é de: {best_threshold}")
print(f"Ganhos por transações aprovadas: R$ {best_income:.2f}")
print(f"Prejuízos com transações fraudulentas aprovadas: R$ {minimum_loss:.2f}")
print(f"Receita gerada com limiar ótimo: R$ {best_revenue:.2f}")

O limiar ótimo encontrado para a amostra é de: 75
Ganhos por transações aprovadas: R$ 433656.06
Prejuízos com transações fraudulentas aprovadas: R$ 168726.99
Receita gerada com limiar ótimo: R$ 264929.07


Para o cálculo dos próximos KPIs, iremos assumir o limiar ótimo de identificação de fraude igual a 75 pontos de score.

**Atenção**: É importante frisar que este valor ótimo será utilizado para análise inicial dos KPIs da amostra, unicamente para verificarmos as métricas do estado de negócio. No momento em que formos comparar com o modelo construído neste projeto, é necessário calcularmos novamente tais métricas utilizando apenas os dados para avaliação/teste.

### *Incoming Pressure*

Para calcular a pressão de entrada de transações fraudulentas utilizaremos a fórmula:
$$
\text{Pressão de Entrada} = \frac{\text{Número de transações fraulentas}}{\text{Total de transações}} \times 100
$$

Como estamos calculando uma métrica de relatório e de diagnóstico, utilizaremos o  real da classificação de fraudes para este KPI.

In [8]:
incoming_pressure = (df["fraude"].sum() / df.shape[0]) * 100
print(f"Taxa de pressão de entrada é de {incoming_pressure}%")

Taxa de pressão de entrada é de 5.0%


Temos o valor de **5%**, podemos utilizar a pressão de entrada para verificar se nossa taxa de aprovação é razoável ou muitro restrita.

### Taxa de Aprovação

Para a taxa de aprovação, utilizaremos todas as transações não marcadas como fraudulentas sobre o total.
$$
\text{Taxa de aprovação} = \frac{\text{Transações Aprovadas}}{\text{Total de transações}} \times 100
$$

In [9]:
approved_count = df.loc[df["score_fraude_modelo"] < best_threshold].shape[0]
approval_rate = (approved_count / df.shape[0]) * 100
print(f"Taxa de aprovação total é de {approval_rate:.2f}%")

Taxa de aprovação total é de 76.51%


Aqui temos uma taxa de aprovação de 76.51%, menor que 80%, o que pode significar uma experiência do usuário não tão satisfatória especialmente para o nível de pressão de entrada que temos menor que 10%.

### Taxa de Declínio

A taxa de declínio pode ser calculada tanto pelo complemento da taxa de aprovação, quanto pela fórmula:
$$
\text{Taxa de declínio} = \frac{\text{Transações Recusadas}}{\text{Total de transações}} \times 100
$$

In [10]:
decline_rate = 100 - approval_rate
print(f"Taxa de declínio total é de {decline_rate:.2f}%")

Taxa de declínio total é de 23.49%


A taxa de declínio pode ser melhor interpretada em conjunto com as métricas de Precisão e Revocação.

### Precisão e Revocação (Recall)

Temos que nosso Verdadeiro Positivo (VP) será dito pelas Transações fraudulentas identificadas e o falso positivo (FP) Transações legítimas identificadas como fraudulentas.

$$
\text{Precisão} = \frac{\text{VP}}{\text{VP} + \text{FP}}
$$



In [11]:
declined_sales = df.loc[df["score_fraude_modelo"] > best_threshold]
vp = declined_sales.loc[(df["fraude"] == 1)].shape[0]
fp = declined_sales.loc[(df["fraude"] == 0)].shape[0]

precision = vp / (vp + fp) * 100
print(f"A precisão do modelo é de {precision:.2f}%")

A precisão do modelo é de 14.04%


Para calcular o recall alterados apenas o FP utilizado pelos Falsos Negativos, ou seja as Transações Fraudulentas que foram classificadas como legítimas. 
$$
\text{Recall} = \frac{\text{VP}}{\text{VP} + \text{FN}}
$$

In [12]:
accepted_sales = df.loc[df["score_fraude_modelo"] < best_threshold]
fn = accepted_sales.loc[(df["fraude"] == 1)].shape[0]

recall = vp / (vp + fn) * 100
print(f"A revocação do modelo é de {recall:.2f}%")

A revocação do modelo é de 63.78%


Com esses ambos valores conseguimos comparar nossa precisão com a taxa de declínio. Temos que o valor de 14.04% é menor que a taxa de declínio de 23.49%, com isso uma grande parcela de transações classificadas como fraudulentas na realidade são legítimas causando perdas desnecessárias no faturamento. A revocação de aproxidamente 63.78% significa que estamos capturando certas fraudes, mas que há um grande espaço para melhoria do modelo.

### Taxa de falsos positivos

Para a taxa de falsos positivos temos quantas transações legítimas foram classificadas como fraudes. Para isso utilzaremos os verdadeiros negativos, transações legítimas que foram classificadas como tal.

$$
\text{Taxa de falsos positivos} = \frac{\text{FP}}{\text{FP} + \text{VN}}
$$

In [13]:
vn = accepted_sales.loc[(df["fraude"] == 0)].shape[0]

false_positive_rate = fp / (fp + vn) * 100
print(f"A taxa de falsos positivos é de {false_positive_rate:.2f}%")

A taxa de falsos positivos é de 20.52%


O que representa uma taxa de falsos positivos alta, imagine que 1 em cada 5 compras em seu cartão, ele irá acusar que você está realizando uma fraude. Tal porcentagem reflete uma dificuldade na experiência do usuário. 

---

Nos próximos notebooks pretendemos utilizar novamente as funções para encontrar o melhor limiar e realizar o cálculo de métricas, porém apenas com a parcela de teste para compararmos com o novo modelo construído, iremos repassar os códigos aqui esboçados neste notebook para o arquivo `src/features/base_metrics.py` para facilitar o reuso em momento posterior.

In [14]:
%cd ..

from src.features.base_metrics import BaseMetrics

bm = BaseMetrics(df, "score_fraude_modelo", "fraude", "valor_compra")

c:\Users\renne\Documents\Estudos Dados\Repositórios\ML\fraud-detection


c:\Users\renne\anaconda3\envs\fraud-detection\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
bm.show_all_metrics()

O limiar ótimo encontrado para a amostra é de: 75
Ganhos por transações aprovadas: R$ 433656.06
Prejuízos com transações fraudulentas aprovadas: R$ 168726.99
Receita gerada com limiar ótimo: R$ 264929.07
Taxa de pressão de entrada é de 5.0%
Taxa de aprovação total é de 76.51%
Taxa de declínio total é de 22.45%
A precisão do modelo é de 14.04%
A revocação do modelo é de 63.78%
A taxa de falsos positivos é de 20.52%
